### Modules

In [ ]:
weber.junior@img.com.br
11 96378-9218

In [2]:
cd ../

C:\Users\luisr\Desktop\Repositories\Data Science Projects\Hackaton COR IV - Centro de Operações do RJ\INCUBAÇÃO


In [5]:
import requests, json, pandas as pd, numpy as np
from matplotlib.path import Path as mpl_path
from IPython.display import clear_output as co

from Modulos.imbalanced_selection import groupConsecutiveFlags

---
# Get prediction alerts from waterbag models api

### Request predictions for region 'Catete'

In [6]:
waterbag_api = 'http://127.0.0.1:5000/'
catete_url = waterbag_api + 'predictions?cluster_id=1'

# api request
catete = pd.DataFrame(requests.get(catete_url).json())

catete.head()

,_id,timestamp,date,time,cluster_id,cluster,range,probability,confidence,label
0,63a1bb325b12cc589ff494f3,2022-12-20 13:40:00.017000,2022-12-20,10:40:00,1,Rua do Catete,1h,0.000909,0.998182,0
1,63a1ba065b12cc589ff494ea,2022-12-20 13:35:00.017000,2022-12-20,10:35:00,1,Rua do Catete,1h,0.000909,0.998182,0
2,63a1b8db5b12cc589ff494e4,2022-12-20 13:30:00.016000,2022-12-20,10:30:00,1,Rua do Catete,1h,0.102764,0.794472,0
3,63a1b7ae5b12cc589ff494de,2022-12-20 13:25:00.010000,2022-12-20,10:25:00,1,Rua do Catete,1h,0.109982,0.780036,0
4,63a1b6835b12cc589ff494d8,2022-12-20 13:20:00.020000,2022-12-20,10:20:00,1,Rua do Catete,1h,0.109982,0.780036,0


### Type conversion

In [7]:
catete['timestamp'] = catete['timestamp'].astype('datetime64[ms]')

catete.set_index('timestamp', inplace=True)
catete.sort_index(inplace=True)

### Operation start

In [8]:
operation_start = catete.index.min()

str(operation_start.date())

'2022-10-19'

### Operation time

In [9]:
pd.datetime.now() - operation_start

<ipython-input-9-18cfd65ff83b>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime.now() - operation_start


Timedelta('61 days 14:19:58.470097')

In [10]:
yprob = catete['probability']
yhat = (yprob >= 0.5).astype('int')
ygroup = groupConsecutiveFlags(yhat)

catete['alert'] = ygroup

### Predicted alerts duration

In [11]:
alert_duration = 5 * ygroup.value_counts().drop(-1).to_frame('duration (min)')

alert_duration.T

,6,0,7,2,8,9,11,1,3,10,4,5
duration (min),165,95,80,70,70,70,70,60,35,20,10,10


### Alerts date and time

In [12]:
start = catete.groupby('alert').first()[['date', 'time']]
end = catete.groupby('alert').last()[['date', 'time']]
end.columns = ['date_end', 'time_end']

alerts = start.join([end[['time_end']], alert_duration]).drop(-1)

alerts

,date,time,time_end,duration (min)
alert,,,,
0,2022-10-31,17:10:00,18:40:00,95.0
1,2022-11-01,13:55:00,14:50:00,60.0
2,2022-11-01,15:20:00,16:25:00,70.0
3,2022-11-01,17:55:00,18:25:00,35.0
4,2022-11-01,18:55:00,19:00:00,10.0
5,2022-11-24,05:45:00,05:50:00,10.0
6,2022-11-24,06:00:00,08:40:00,165.0
7,2022-11-28,13:45:00,15:00:00,80.0
8,2022-11-28,15:15:00,16:20:00,70.0


---
# Get *closed* waterbag events from Sistema Comando

### Set api urls and request parameters 

In [13]:
comando_api = 'https://api.dados.rio/v2/adm_cor_comando/'
operation_events_url = comando_api + 'ocorrencias'

days_30 = pd.offsets.Day() * 30
now = pd.datetime.now()

n_requests = int(np.ceil((now - operation_start) / (days_30)))

<ipython-input-13-1300566dc13c>:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  now = pd.datetime.now()


### Request events from comando api
#### From operation start time until now, in periods of 30 days

##### Obs: This api endpoint returns a obj containing a list of objects each containing a list of events (objs) corresponding to each day of the period of the request.

In [157]:
start = operation_start
n_attempts = 5

def dict_df_map(obj):
    if 'eventos' in obj.keys():
        return pd.DataFrame((obj['eventos']))
    else:
        return pd.DataFrame([obj])
    
events_df = []
for n in range(n_requests):
    events_res = {}
    if n != 0: start = start + days_30
    start_str = str(start)[:-6] + '0'
    end = start + days_30
    end_str = str(end)[:-6] + '0'
    rqst_url = operation_events_url + f'?inicio={start_str}&fim={end_str}'
    j = 1
    while 'eventos' not in events_res.keys():
        co(True); print('Periods requested:', n + 1, '/', n_requests,'\nRetries:', str(j), '/', n_attempts)
        events_res = requests.get(rqst_url).json()
        if j>=n_attempts: break
    events = pd.concat(list(map(dict_df_map, events_res['eventos'])))
    events_df.append(events)

events_df = pd.concat(events_df).reset_index(drop=True)

print('Done! Status: Success')

Periods requested: 3 / 3 
Request retries: 1 / 5


### Data cleaning

In [158]:
events_df['inicio'] = events_df['inicio'].astype('datetime64[ms]')
events_df['fim'] = events_df['fim'].astype('datetime64[ms]')

### Events temporal description

In [159]:
events_df['inicio'].describe(datetime_is_numeric=True)[['count', 'min', 'max']]

count                   3712
min      2022-10-19 05:59:54
max      2022-12-20 13:15:06
Name: inicio, dtype: object

### Get events inside the model's region of influence

#### Get the region polygon

In [160]:
polygons_geojson_path = '../ACELERAÇÃO/Dados/Clusters/polygons_micro.geojson'

# Load custers polygons geojson data
polygons_geojson = json.loads(open(polygons_geojson_path, 'r').read())

# Get polygons dict from geojson
polygons = {}
for poly in polygons_geojson['features']:
    if poly['properties']['sublabel'] != -1:
        polygons[poly['properties']['sublabel']] = poly['geometry']['coordinates'][0]

#### Get events inside polygon

In [162]:
#### Get events points as array
points = np.array(list(map(tuple, events_df[['longitude', 'latitude']].values)))

cluster_id = 1
poly = polygons[cluster_id]

# Get events inside polygon
points_msk = mpl_path(poly).contains_points(points)
events_poly = events_df[points_msk]

events_poly

,tipo,pop_id,bairro,latitude,inicio,titulo,fim,prazo,descricao,informe_id,gravidade,id,longitude,status
743,PRIMARIO,5.0,Catete,-22.927022,2022-11-01 15:39:00,Bolsão d'água em via ( CAM 313 ),2022-11-01 17:58:04,CURTO,"R. do Catete, Alt. R. Silveira Martins - Catete",83611,BAIXO,83610,-43.176622,FECHADO
2155,PRIMARIO,31.0,Glória,-22.925695,2022-11-28 13:33:46,Alagamento (Cam 313),2022-11-28 16:17:42,CURTO,"Rua do Catete, Alt. R. Silveira Martins - Catete",85187,MEDIO,85186,-43.176198,FECHADO
2158,PRIMARIO,5.0,Glória,-22.926966,2022-11-28 13:40:46,Bolsão d'água em via (Cam 268),2022-11-28 15:15:56,CURTO,R. do Catete - Largo do Machado,85190,BAIXO,85189,-43.176611,FECHADO
3569,PRIMARIO,5.0,Glória,-22.926966,2022-12-18 04:32:46,Bolsão d'água em via ( CAM 313 ),2022-12-18 06:04:39,CURTO,R. do Catete - Alt. Rua Silveira Martins - Glória,86492,BAIXO,86491,-43.176611,FECHADO


### Get events by date

In [163]:
'2022-10-31', '2022-11-01', '2022-11-24', '2022-11-28', '2022-12-18'

events_df.set_index('inicio')['2022-12-20']#.set_index('pop_id')[5]

<ipython-input-163-8beccfd874be>:3: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  events_df.set_index('inicio')['2022-12-20']#.set_index('pop_id')[5]


,tipo,pop_id,bairro,latitude,titulo,fim,prazo,descricao,informe_id,gravidade,id,longitude,status
inicio,,,,,,,,,,,,,
2022-12-20 01:17:09,PRIMARIO,14.0,São Cristóvão,-22.889825,Fogo em fiação,2022-12-20 04:51:54,CURTO,"Tv. Doná Darcí Vargas, Alt. nº 5 - São Cristóvão",86584,BAIXO,86583,-43.231279,FECHADO
2022-12-20 01:55:02,PRIMARIO,2.0,Engenho Novo,-22.919308,AC VP X Poste,2022-12-20 05:21:40,CURTO,"Estr. Grajaú-Jacarepaguá, Alt. Km 3,5 - Engenh...",86585,BAIXO,86584,-43.284373,FECHADO
2022-12-20 02:33:52,PRIMARIO,1.0,Irajá,-22.823040,Tombamento CM,2022-12-20 02:59:54,CURTO,"Av. Brasil, Alt. Passarela 24 - Irajá - Sent. ...",86586,BAIXO,86585,-43.327911,FECHADO
2022-12-20 02:38:09,PRIMARIO,2.0,Manguinhos,-22.885643,Capotamento TX,2022-12-20 03:01:54,CURTO,"Av. Brasil, Alt. Passarela 5 - Benfica - Sent....",86587,BAIXO,86586,-43.234455,FECHADO
2022-12-20 03:06:03,PRIMARIO,5.0,Barra da Tijuca,-23.006711,Bolsão d'água em via (Cam. 3133),2022-12-20 09:37:41,CURTO,"Av. Armando Lombardi, Alt. Est. BRT Jardim Oce...",86588,BAIXO,86587,-43.312489,FECHADO
2022-12-20 04:11:10,PRIMARIO,2.0,Coelho Neto,-22.830703,AC VP X Mureta (Cam. 153),2022-12-20 05:47:55,CURTO,"Av. Brasil, Alt. R. João Paulo - Coelho Neto -...",86590,BAIXO,86589,-43.350638,FECHADO
2022-12-20 04:28:37,PRIMARIO,1.0,Guaratiba,-22.969733,Enguiço VP,2022-12-20 05:15:58,CURTO,"Av. Dom João VI, Alt. de Guaratiba - Sent. Sa...",86591,BAIXO,86590,-43.632156,FECHADO
2022-12-20 04:30:58,PRIMARIO,2.0,Recreio dos Bandeirantes,-23.017588,AC 2 VPs,2022-12-20 05:20:50,CURTO,"Av. das Américas, Alt. Est. BRT Gláucio Gil - ...",86592,BAIXO,86591,-43.480610,FECHADO
2022-12-20 05:00:36,PRIMARIO,2.0,Penha Circular,-22.839589,Capotamento UT,2022-12-20 05:03:58,CURTO,"Av. Vicente de Carvalho, Alt. Praça do Carmo -...",86593,BAIXO,86592,-43.296462,FECHADO
